In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit
from scipy.stats import chi2

# Lettura dei dati


In [2]:
# salvo i dati in un dataframe

tempi_approx = [0, 5, 12, 21, 28, 36, 43, 50, 60, 72, 84, 97]
data = {}
data_temp = {}

for i in tempi_approx:
    filename = f'./data/t{i}.TXT'
    read = pd.read_csv(filename, delimiter='\t', skiprows=19)
    λ = read.iloc[:, 0].to_numpy()
    A = read.iloc[:, 1].to_numpy()
    data_temp[i] = pd.DataFrame({'λ': λ, 'A': A})

# rinomino i tempi in modo che siano uguali a quelli reali
tempi = [0, 5, 12.7, 21, 28.48, 36.1, 43.33, 50.47, 60, 70.07, 82.33, 97.42]
for approx, real in zip(tempi_approx, tempi):
    data[real] = data_temp[approx]
    tempi = np.array(tempi)

# Rimuovo i primi 50 dati per tempo zero perchè abbiamo preso misure in più
min_length = min(len(data[t]['λ']) for t in tempi)
if 0 in data:
    data[0] = data[0].iloc[50:].reset_index(drop=True)

# Fit parabolici

<h3> Dati iniziali

In [3]:
colors = px.colors.sample_colorscale('Dense', [i / 11 for i in range(12)])
fig = go.Figure()

for idx, (key, df) in enumerate(data.items()):
    fig.add_trace(go.Scatter(x = df['λ'], y = df['A'], mode = 'lines', name = f't = {key} ns', 
                             line = dict(color = colors[idx])))

    # error_y=dict( 
    #     type='data', 
    #     array = sigmay, 
    #     color = colors[2 * i],
    #     thickness = 1.5, 
    #     width = 3, 
	# 	)

fig.update_layout(
    xaxis_title='λ',
    yaxis_title='A',
    legend_title='Tempi',
    template = 'plotly_white'
)

fig.show()

<h3> Errori

In [ ]:
σ_λ = 0.0753  #(nm) dichiarato dal costruttore

# errore percentuale A
σ_A_perc = 0.001719

# circa 20 secondi persi a spipettare tutte le volte
err_t = 0.33  # (min)

NOTA: moltiplico per 2 perché mi pare una cosa sensata da fare sottraendo le curve, no?

<h3> Accoppio le code

In [5]:
# accoppio le code e creo un dataframe con i dati aggiornati

minimi = np.array([df['A'].min() for df in data.values()])
min_minimo = min(minimi) 
offset = minimi - min_minimo  # Calculate the offset for each time point

# Initialize data_agg as a dictionary
data_agg = {}

for idx, i in enumerate(tempi):
    data_agg[i] = data[i].copy()  # Create a copy of the original dataframe
    data_agg[i]['A'] = data[i]['A'] - offset[idx]  # Adjust the 'A' values using the corresponding offset



In [6]:
fig_agg = go.Figure()

for idx, (key, df) in enumerate(data_agg.items()):
    fig_agg.add_trace(go.Scatter(x=df['λ'], y=df['A'], mode='lines', name=f't = {key} ns', 
                             line=dict(color=colors[idx])))

    # error_y=dict( 
    #     type='data', 
    #     array = sigmay, 
    #     color = colors[2 * i],
    #     thickness = 1.5, 
    #     width = 3, 
	# 	)

fig_agg.update_layout(
    xaxis_title='λ',
    yaxis_title='A',
    legend_title='Tempi',
    template = 'plotly_white'
)

fig_agg.show()

<h3> Sottraggo le curve e rappresento

In [7]:
# faccio la sottrazione tra le curve

data_diff = {}

for idx, i in enumerate(tempi):
    data_diff[i] = data_agg[i].copy()  # Create a copy of the original dataframe
    data_diff[i]['A'] = data_agg[0]['A'] - data_agg[i]['A'] 


In [8]:
fig_diff = go.Figure()

for idx, (key, df) in enumerate(data_diff.items()):
    fig_diff.add_trace(go.Scatter(x=df['λ'], y=df['A'], mode='lines', name=f't = {key} ns', 
                             line=dict(color=colors[idx])))

    # error_y=dict( 
    #     type='data', 
    #     array = sigmay, 
    #     color = colors[2 * i],
    #     thickness = 1.5, 
    #     width = 3, 
	# 	)

fig_diff.update_layout(
    xaxis_title='λ',
    yaxis_title='A',
    legend_title='Tempi',
    template = 'plotly_white'
)

fig_diff.show()

<h3> Definisco la parabola e trovo i massimi. Salvo i subset attorno ai massimi

In [9]:
# Definizione della parabola per il fit
def parabola(x, a, xV, yV):
    return a * (x - xV)**2 + yV

r = 5

# Trova i massimi delle curve sottratte e salva i subset attorno ai massimi
picchi = {}
for key, df in data_diff.items():
    if key == 0:
        continue
    max_index = df['A'].idxmax()
    start = max(0, max_index - r)
    end = min(len(df), max_index + r + 1)
    subset = df.iloc[start:end]
    picchi[key] = subset

<h3> Faccio l'effettivo fit parabolico, e plotto dati iniziali, parabole e vertici

In [10]:
fit_results = []
yV_err = []

fig_parabolic_fit = go.Figure()

# Dati sottratti
for idx, (key, df) in enumerate(data_diff.items()):

    fig_parabolic_fit.add_trace(go.Scatter(
        x=df['λ'],
        y=df['A'],
        mode='lines',
        name=f'Tempo {key}',
        line=dict(dash='dot', color=colors[idx], width=2),
        showlegend=False
    ))

# picchi, fit parabolico e errori
for key, subset in picchi.items():

    if key == 0:
        continue
    x_data = subset['λ']
    y_data = subset['A']


    # -------------- ERRORI e FIT ----------------

    # iterazione 0
    popt, pcov = curve_fit(parabola, x_data, y_data, p0=[-1, x_data.mean(), y_data.max()])
    a, xV, yV = popt
    err_a, err_xV, err_yV = np.sqrt(np.diag(pcov))

    # iterazione 1
    dydl = np.abs(2 * a * (x_data - xV))
    σ_y_prop = dydl * σ_λ
    σ_A_1 = σ_A_perc * y_data
    σ_y = np.sqrt(σ_y_prop ** 2 + σ_A_1 ** 2)

    popt, pcov = curve_fit(parabola, x_data, y_data, p0 = [a, xV, yV], sigma = σ_y, absolute_sigma = True)
    
    a, xV, yV = popt
    err_a, err_xV, err_yV = np.sqrt(np.diag(pcov))

   # ---------------------------------------


    x_fit = np.linspace(x_data.min(), x_data.max(), 100)
    y_fit = parabola(x_fit, *popt)

    yV_err.append(err_yV)

    # Aggiungi i risultati alla lista
    fit_results.append({
        'Tempo (ns)': key,
        'a': a,
        'err a': err_a,
        'xV': xV,
        'err xV': err_xV,
        'yV': yV,
        'err yV': err_yV
    })
    
    # Aggiungi il fit parabolico
    fig_parabolic_fit.add_trace(go.Scatter(
        x = x_fit,
        y = y_fit,
        mode = 'lines',
        name = f'Fit Tempo {key}',
        line = dict(color=colors[7]),
        showlegend = False
    ))

    # Aggiungi il punto del picco
    fig_parabolic_fit.add_trace(go.Scatter(
        x = [popt[1]],  # xV
        y = [popt[2]],  # yV
        mode = 'markers',
        name = f'Picco Tempo {key}',
        marker = dict(size = 3, color = colors[10], symbol = 'circle'),
        error_y = dict(
            type = 'data',
            array = [err_yV],
            visible = True,
            color = colors[10],
            thickness = 1.5,
            width = 3
        ),
        # error_x = dict(
        #     type = 'data',
        #     array = [err_xV],
        #     visible = True,
        #     color = colors[10],
        #     thickness = 1.5,
        #     width = 3
        # ),
        showlegend = False
    ))

fig_parabolic_fit.update_layout(
    xaxis_title='λ',
    yaxis_title='A',
    legend_title='Legenda',

    template='plotly_white'
)

fig_parabolic_fit.show()

<h3> Stampo i parametri ricavati dai fit parabolici con i relativi errori

In [11]:
# Creazione del DataFrame
fit_results_df = pd.DataFrame(fit_results)

# Stampa della tabella
print(fit_results_df.round(5))

    Tempo (ns)        a    err a         xV   err xV       yV   err yV
0         5.00 -0.00007  0.00000  527.42745  0.03687  0.01725  0.00002
1        12.70 -0.00012  0.00000  526.29755  0.04006  0.03367  0.00003
2        21.00 -0.00014  0.00000  525.59812  0.03866  0.04555  0.00004
3        28.48 -0.00017  0.00000  525.98319  0.03921  0.05217  0.00004
4        36.10 -0.00020  0.00001  526.01741  0.04144  0.06408  0.00005
5        43.33 -0.00030  0.00001  526.32190  0.03504  0.07441  0.00007
6        50.47 -0.00032  0.00001  526.56183  0.03506  0.08287  0.00007
7        60.00 -0.00025  0.00001  526.51540  0.04969  0.08473  0.00007
8        70.07 -0.00032  0.00001  526.60002  0.03722  0.09356  0.00008
9        82.33 -0.00034  0.00001  526.49427  0.03780  0.10042  0.00009
10       97.42 -0.00031  0.00001  526.39354  0.04968  0.11012  0.00009


<h3> Chi quadro e pvalue per ogni fit

In [12]:
chi_squared_results = []

for key, subset in picchi.items():
    if key == 0:
        continue

    x_data = subset['λ']
    y_data = subset['A']

    # -------------- ERRORI e FIT ----------------
    popt, _ = curve_fit(parabola, x_data, y_data, p0=[-1, x_data.mean(), y_data.max()])
    a, xV, yV = popt
    dydl = np.abs(2 * a * (x_data - xV))
    σ_y_prop = dydl * σ_λ

    # Fit pesato finale
    popt, _ = curve_fit(parabola, x_data, y_data, p0=popt, sigma=σ_y, absolute_sigma=True)

    y_fit = parabola(x_data, *popt)
    residuals = (y_data - y_fit) / σ_y
    chi_squared = np.sum(residuals**2)
    dof = len(y_data) - len(popt)
    chi_squared_reduced = chi_squared / dof
    p_value = 1 - chi2.cdf(chi_squared, dof)

    chi_squared_results.append({
        'Tempo (ns)': key,
        'Chi-squared-reduced': chi_squared_reduced,
        'p-value': p_value
    })

# Creazione del DataFrame
chi_squared_results_df = pd.DataFrame(chi_squared_results)
# Stampa della tabella
print(chi_squared_results_df.round(5))

    Tempo (ns)  Chi-squared-reduced  p-value
0         5.00              9.06853  0.00000
1        12.70             16.25266  0.00000
2        21.00             17.09674  0.00000
3        28.48             15.01455  0.00000
4        36.10             13.61506  0.00000
5        43.33              5.34303  0.00000
6        50.47              5.96131  0.00000
7        60.00             19.16811  0.00000
8        70.07              2.66022  0.00644
9        82.33              7.17975  0.00000
10       97.42             14.47111  0.00000


# Tempo medio di ingresso della rodamina nella pastiglia

<h3> Funzione per il fit dei picchi

In [13]:
def delta_abs (t, a, b, k):
    return b * (1 - a * np.exp(-k * t))

<h3> Fit esponenziale

In [14]:
# # propagazione errori
# def propagate_errors(t, a, b, k, sigma_a, sigma_b, sigma_k):
#     # Derivate parziali
#     d_delta_abs_da = -b * np.exp(-k * t)
#     d_delta_abs_db = 1 - a * np.exp(-k * t)
#     d_delta_abs_dk = b * a * t * np.exp(-k * t)

#     # Propagazione degli errori
#     sigma_delta_abs = np.sqrt((d_delta_abs_da * sigma_a) ** 2 + (d_delta_abs_db * sigma_b) ** 2 + (d_delta_abs_dk * sigma_k) ** 2)
#     return sigma_delta_abs

# # fit esponenziale

# popt_exp, pcov_exp = curve_fit(delta_abs, tempi[1:], minimi[1:], p0 = [1, 1, 0.01])
# a, b, k = popt_exp

# # Fit esponenziale
# popt, pcov = curve_fit(delta_abs, tempi[1:], minimi[1:], p0=[1, 1, 0.01])
# a, b, k = popt
# err_a, err_b, err_k = np.sqrt(np.diag(pcov))
# # Calcolo degli errori propagati
# sigma_delta_abs = []
# for t in tempi[1:]:
#     sigma = propagate_errors(t, a, b, k, err_a, err_b, err_k)
#     sigma_delta_abs.append(sigma)
# sigma_delta_abs = np.array(sigma_delta_abs)

# value = delta_abs(0, a, b, k)
# error = propagate_errors(0, a, b, k, err_a, err_b, err_k)
# final_value = final_val(value, error)
# final_error = propagate_errors(0, a, b, k, err_a, err_b, err_k)
# # Stampa dei risultati
# print(f"Valore finale: {final_value}")

In [15]:
t = np.array(list(picchi.keys()))  # Tempi
A = np.array([df['A'].max() for df in picchi.values()])  # Ampiezze

a, b, k = 1, 0.1, 0.01  # p0

# Calcolo derivata ∂y/∂t = |a * b * k * exp(-k * t)| per propagare l'errore su t
dydt = np.abs(a * b * k * np.exp(-k * t))
dy_prop = dydt * err_t  # Errore su y derivante da err_t

# Errore da precedente fit su y (terzo parametro)
erryV = np.sqrt(np.diag(pcov))[2]  # Devi avere già fatto un fit precedente

yV_err = np.array(yV_err)  # Converti in array numpy
# Somma in quadratura gli errori
sigma1 = np.sqrt(dy_prop**2 + yV_err**2)

# Stima iniziale dei parametri (a, b, k)
p0 = [1, 0.1, 0.01]

# Fit finale con incertezze corrette
popt1, pcov1 = curve_fit(
    delta_abs, t, A, p0=p0,
    sigma=sigma1, absolute_sigma=True,
    maxfev=5000
)

# Incertezze sui parametri
perr1 = np.sqrt(np.diag(pcov1))

# Output
print(f"Parametri stimati:")
print(f"a = {popt1[0]:.4f} ± {perr1[0]:.4f}")
print(f"b = {popt1[1]:.4f} ± {perr1[1]:.4f}")
print(f"k = {popt1[2]:.4f} ± {perr1[2]:.4f}")


Parametri stimati:
a = 0.9278 ± 0.0080
b = 0.1372 ± 0.0026
k = 0.0157 ± 0.0007


In [16]:
# Crea una figura per i picchi e il fit
fig_fit = go.Figure()

t_fit = np.linspace(t.min(), t.max(), 500)  # Intervallo di tempo per il fit
A_fit = delta_abs(t_fit, *popt1)  # Valori del fit

# Aggiungi la curva del fit
fig_fit.add_trace(go.Scatter(
    x=t_fit,
    y=A_fit,
    mode='lines',
    name='Fit',
    line=dict(color = colors[3], width=2),
    showlegend=False
))

# Dati originali (picchi)
picchi_values = np.array([df['A'].max() for df in picchi.values()])
picchi_times = np.array(list(picchi.keys()))

# Aggiungi i picchi
fig_fit.add_trace(go.Scatter(
    x=picchi_times,
    y=picchi_values,
    mode='markers',
    name='Picchi',
    marker=dict(color=colors[10], size=3),
    error_y=dict(
        type='data',
        array=sigma1,
        visible=True,
        color=colors[10],
        thickness=1.5,
        width=3
    ),
    showlegend=False
))

fig_fit.update_layout(
    xaxis_title='t (ns)',
    yaxis_title='ΔA',
    legend_title='Legenda',
    width=600,
    template='plotly_white'
)

fig_fit.show()

# Parametri del fit con errore
print(f"Parametri: a = {popt1[0]:.4f} ± {perr1[0]:.4f}, b = {popt1[1]:.4f} ± {perr1[1]:.4f}, k = {popt1[2]:.4f} ± {perr1[2]:.4f}")
print('\n')
print('Tempo impiegato (in media) dalla rodamina per essere assorbita dalla pastiglia:')
print(f't = ({1/popt1[2]:.4f} ± {perr1[2]/(popt1[2]**2):.4f}) ns')


Parametri: a = 0.9278 ± 0.0080, b = 0.1372 ± 0.0026, k = 0.0157 ± 0.0007


Tempo impiegato (in media) dalla rodamina per essere assorbita dalla pastiglia:
t = (63.8408 ± 2.9112) ns


<h3> Chi quadri e pvalue

In [17]:
res1 = (A - delta_abs(t, *popt1)) / sigma1  # residui normalizzati
chi_squared1 = np.sum(res1 ** 2)
chi_squared_reduced1 = chi_squared1 / (len(A) - len(popt1))  # Chi quadro ridotto
dof1 = len(A) - len(popt1)  # gradi di libertà
p_value1 = 1 - chi2.cdf(chi_squared1, dof1)

print(f"Chi quadro: {chi_squared1:.4f}")
print(f"Chi quadro ridotto: {chi_squared_reduced1:.4f}")
print(f"p-value: {(p_value1 * 100):.2f} %")

if p_value1 >= 0.05:
    print("LETSGOSKIIIIIIIIIIII")
else: print("NOOOOOOOOOOOOOOOOOOOOOOO")


Chi quadro: 64.0641
Chi quadro ridotto: 8.0080
p-value: 0.00 %
NOOOOOOOOOOOOOOOOOOOOOOO


# ANNOTAZIONI

- Ho tolto il fondo nel fit finale perché ha errore zero, e questa cosa manda in palla il codice
- CHE SCHIFO!!!!!!!!!!!!!!!!!!!!!!!